In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mercedes-benz-greener-manufacturing/train.csv.zip
/kaggle/input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip
/kaggle/input/mercedes-benz-greener-manufacturing/test.csv.zip


In [2]:
pd.options.display.max_columns = 1000
train = pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/train.csv.zip")
test = pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/test.csv.zip")

In [3]:
all_data = pd.concat([train,test])
all_data2 = all_data.drop(["y"],1)
# y값이 예측해야하는 target컬럼

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [5]:
# le.classes_
# le

In [6]:
# 데이터타입이 object인 컬럼만 뽑아
cate_cols = all_data2.columns[all_data2.dtypes == 'object']
for i in cate_cols:
    all_data2[i] = le.fit_transform(list(all_data2[i]))
    # label encoding 해준다.

In [7]:
# all data -> Train, Test로 분리
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

In [8]:
from xgboost import XGBRegressor
# xgb = XGBRegressor(learning_rate=0.1)
# xgb = XGBRegressor(learning_rate=0.1,max_depth=3,)
# xgb = XGBRegressor(learning_rate=0.1,max_depth=3,colsample_bytree=0.5)
# xgb = XGBRegressor(learning_rate=0.1,max_depth=3,colsample_bytree=0.5,n_estimators=80)
## max_depth = 6보다 3, n_estimator d
## subsample = 기본값 1로 설정되어있다.

# lr을 0.01로 해서 한 결과가 더 성능이 좋게 나온다.
xgb = XGBRegressor(learning_rate=0.01,max_depth=3,colsample_bytree=0.5,n_estimators=800)
# 앙상블 효과를 줘서 좋은 결과를 가져오도록 하였다. (n_estimator 800번)
xgb.fit(train2, train["y"])
result2 = xgb.predict(test2)

In [9]:
# learning_rate=0.1,max_depth=3,colsample_bytree=0.5,subsample = 1, n_estimators=80

# 나머지를 다 정해준다음, learning_rate를 마지막에 설정해주는것이 좋다.
# 처음 tuning할 떄는 learning rate를 0.1로 나머지들 값을 찾아준 후, lr를 바꿔주는 것이 좋다.
# max_depth : decision tree에서 노드의 깊이를 조절해주는 것, default값이 기본적으로 6이다.
# colsample_bytree : =1 - 칼럼을 다 사용, =0.8 - 칼럼을 80%만 사용한다. 1보다 0.8, 0.9정도로 놓고 해야 과적합 줄여주고, 앙상블 효과를 주기 때문에 성능이 높아지는 경우가 많다
# subsample : =1 모든 데이터 활용(default), 0.8, 0.9 이런식으로 설정 가능
# n_estimators : default 100이며, 의사결정나무를 100번 실행한다 (이전 실행 결과를 다음 모듈 input으로 활용하여 100번 실행)

# 모두 과적합을 방지하기 위한 장치들

In [10]:
# 이제 Learning rate를 조절해서 결과를 확인해보자.
# Learning rate를 바꿀때, n_estimator도 비례하게 변경해줘야한다.
# xgb = XGBRegressor(learning_rate=0.1,max_depth=3,colsample_bytree=0.5,n_estimators=80)
# xgb = XGBRegressor(learning_rate=0.01,max_depth=3,colsample_bytree=0.5,n_estimators=800)
# lr이 0.1일때, 80번으로, 0.01일때 800번으로 이렇게 반비례하게 설정 변환해줘야한다.

In [11]:
sub = pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip")
sub['y'] = result2
sub.to_csv("sub_0824.csv",index = 0)

In [12]:
sub

,ID,y
0,1,83.972076
1,2,102.263741
2,3,84.074417
3,4,78.722122
4,5,113.861000
...,...,...
4204,8410,102.140999
4205,8411,92.601723
4206,8413,92.706245
4207,8414,111.808594


In [13]:
# random_seed 를 활용해 여러번 돌려 앙상블 효과를 줘서 더 성능을 높여줄 수 있다.
# random state 값을 고정하여 -> xgboost에서 램던하게 변화하는 값이 있는데 해당 값을 고정해준다.
random_seed = [1111,2222,3333,4444,5555,6666,7777,8888,9999,1000]
result = 0

for i in random_seef[:7]:
    xgb = XGBRegressor(learning_rate=0.01, max_depth=3,colsample_bytree=0.5,n_estimators=800,random_state=i)
    xgb.fit(train2, train["y"])
    result_A += 

SyntaxError: invalid syntax (<ipython-input-13-fc5e77bcbee3>, line 9)

In [14]:
# Greed search 
# random search
# 베이지안 옵티마이제이션 - 요즘 많이 활용
# 이런 방법들을 활용하여 적절한 parameter값을 자동으로 찾을 수 있다.
#https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
from bayes_opt import BayesianOptimization
import xgboost as xgb
def xgb_eval(max_depth,colsample_bytree,subsample):
    params = {
        'eval_metric': 'rmse',
        'max_depth' : int(max_depth),
        'colsample_bytree' : colsample_bytree,
        'subsample' : subsample,
#         'n_estimators' : int(n_estimators),
        'learning_rate' : 0.1
    }
    xgb_train = xgb.DMatrix(np.array(train2),train["y"]) # train칼럼명 겹칠수 있어서 np.array로 묶음
    cv_result = xgb.cv(params,xgb_train, #교차검증 점수 이렇게 바로 가져올 수 있음(파이썬 레퍼)
                       num_boost_round=1000, #num_boost_round는 어차피 하다가 멈추니까 엄청 높게 하는게 좋음. 여기선 시간관계상
                      early_stopping_rounds=10,
                      nfold=4)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]
xgb_opt = BayesianOptimization(xgb_eval,{"max_depth":(3,15),
                                        "colsample_bytree":(0.5,1),
                                        "subsample":(0.5,1)})
#                                         "n_estimators":(60,200)})
xgb_opt.maximize(n_iter=10,init_points=3)

|   iter    |  target   | colsam... | max_depth | subsample |
-------------------------------------------------------------
|  1        | -8.651    |  0.6537   |  10.74    |  0.5098   |
|  2        | -8.444    |  0.6255   |  7.312    |  0.762    |
|  3        | -8.323    |  0.9681   |  3.545    |  0.6929   |
|  4        | -8.521    |  0.6167   |  9.875    |  0.6039   |
|  5        | -8.347    |  0.8951   |  4.344    |  0.7072   |
|  6        | -8.961    |  1.0      |  15.0     |  1.0      |
|  7        | -8.31     |  0.5      |  3.0      |  1.0      |
|  8        | -8.328    |  0.5      |  3.0      |  0.5      |
|  9        | -8.331    |  1.0      |  3.0      |  1.0      |
|  10       | -8.31     |  0.5      |  3.842    |  1.0      |
|  11       | -8.328    |  0.5      |  3.807    |  0.5      |
|  12       | -8.31     |  0.5      |  3.409    |  1.0      |
|  13       | -8.358    |  0.5      |  5.177    |  1.0      |
